# we need to predict the happiness score considering all the other factors mentioned in the dataset

## LinearRegressionModel

In [ ]:
#lets import necessary library
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings('ignore')

In [ ]:
#read csv file
data = pd.read_csv('happiness_score_dataset.csv')
data.head()

In [ ]:
#check columns
data.columns

In [ ]:
# how big is the dataset
data.shape

In [ ]:
# check the null values
data.isnull().sum()

In [ ]:
# statistics of the dataset
data.describe()

In [ ]:
# Documentation
#1) there is no null values
#2) after looking into all the data i think its ok 
#3) i will remove happiness rank column because it is useless and it act like a numbering to row


In [ ]:
#spliting the data into dependent variable and independent variable
X=data.drop(columns=['Country', 'Region', 'Happiness Rank', 'Happiness Score'],axis=1)
y=data['Happiness Score']

In [ ]:
# lets see how data is distributed
plt.figure(figsize=(20,25) , facecolor='yellow')
plotnumber = 1

for column in X:
    
    if plotnumber <= 9:
        ax=plt.subplot(3,3,plotnumber)
        sns.distplot(data[column])
        plt.xlabel(column , fontsize=10)
        
    plotnumber += 1
plt.show()

In [ ]:
#skewness in terms of number
X.skew().sort_values(ascending=False)

In [ ]:
#applying Power Transformation beacuse our dataset is small and we want to keep outlier
from sklearn.preprocessing import power_transform , PowerTransformer
pt=PowerTransformer()

In [ ]:
#transformed data
X_scaled=pt.fit_transform(X)
X_scaled

In [ ]:
#again checking the skewness
pd.DataFrame(X_scaled,columns=X.columns).skew().sort_values(ascending=False)

In [ ]:
#we will convert numpy array into dataframe
X_new=pd.DataFrame(X_scaled,columns=X.columns)

In [ ]:
X_new.skew().sort_values(ascending=False)

In [ ]:
#y=data['Happiness Score']

In [ ]:
#Best Random State
MaxAccu=0
MaxRS=0

for i in range (0,11):
    X_train,X_test,y_train,y_test=train_test_split(X_new,y,test_size=0.25,random_state=i)
    regression=LinearRegression()
    regression.fit(X_train,y_train)
    
    pred=regression.predict(X_train)
    training=regression.score(X_train,y_train)
    print ('Training Score' , training , 'RandomState' ,i)
    
    y_pred=regression.predict(X_test)
    testing=regression.score(X_test,y_test)
    print ('Testing Score' , testing , 'RandomState' ,i)
    print('\n')
    
    
    if testing>MaxAccu:
        MaxAccu=testing
        MaxRS=i
        print('MAXINING TESTING SCORE' , MaxAccu , 'ON RANDOM STATE OF' , i)
        

In [ ]:
print('Best Accuracy is ' , MaxAccu , 'On Random State' , MaxRS)

In [ ]:
#splliting our data into train test split and randomstate 6
X_train,X_test,y_train,y_test=train_test_split(X_new,y,test_size=0.25,random_state=6)

In [ ]:
#Training the data on Linear Regression Model
regression=LinearRegression()
regression.fit(X_train,y_train)

In [ ]:
#training score
regression.score(X_train,y_train)  

In [ ]:
#testing score
regression.score(X_test,y_test)      

In [ ]:
y_pred=regression.predict(X_test)
y_pred

In [ ]:
# how our pred vs actual look like in graph 
plt.scatter(y_test , y_pred)
plt.xlabel('Actual happiness score')
plt.ylabel('Predicted happiness score')
plt.title('Actual VS Model Prediction')
plt.show()

In [ ]:
#error in terms of numbers
from sklearn.metrics import mean_squared_error,mean_absolute_error

In [ ]:
# again predict 
y_pred = regression.predict(X_test)

In [ ]:
#MAE
mean_absolute_error(y_test,y_pred)

In [ ]:
#MSE
mean_squared_error(y_test,y_pred)

In [ ]:
#RMSE
np.sqrt(mean_squared_error(y_test,y_pred))

In [ ]:
#now we sucessfully created model and our score is great so we check it for overfitting 
#to check overfitting we have to use regularization technique

# in regularization we have 3 type but we use only 2 



# 3 types are 1) LASSO (L1 form) 2)RIDGE (L2 FORM)  3)ELASTIONET

In [ ]:
#BASICALLY LASSO N RIDGE WILL GIVE YOU BEST LEARNING RATE THAT IS ALPHA SCORE AND TRAIN MODEL ACCORDING TO IT

In [ ]:
#import library
from sklearn.linear_model import Ridge,Lasso,RidgeCV,LassoCV

# LASSO MODEL

In [ ]:
##### LASSO MODEL######

lasscv = LassoCV(alphas = None , max_iter = 100, normalize = True)

lasscv.fit(X_train , y_train)

In [ ]:
# best aplha parameter
alpha = lasscv.alpha_
alpha

In [ ]:
# now we have best parametr noe train according to it
lasso_reg = Lasso(alpha)
lasso_reg.fit(X_train,y_train)

In [ ]:
# now check r2 score
lasso_reg.score(X_test,y_test)

# RIDGE MODEL

In [ ]:
############ RIDGE MODEL#########

ridgecv = RidgeCV(alphas = np.arange(0.001,0.1,0.01), normalize = True)
ridgecv.fit(X_train , y_train)


In [ ]:
# best aplha parameter
alpha = ridgecv.alpha_
alpha

In [ ]:
# now we have best parametr noe train according to it
ridge_reg = Ridge(alpha)
ridge_reg.fit (X_train,y_train)

In [ ]:
# now check r2 score
ridge_reg.score(X_test,y_test)

In [ ]:
# after seeing LASSO and RIDGE score thus we can conclude that our model was not overfitted model
## for conformation lets see cross validation of the model

# Cross Vaildation

In [ ]:
#Cross Vaildation
training=regression.score(X_train,y_train)
testing=regression.score(X_test,y_test)

from sklearn.model_selection import cross_val_score
for j in range(2,10):
    cv_score=cross_val_score(regression,X_new,y,cv=j)
    cv_mean=cv_score.mean()
    print(f'At cross fold {j} the cv score is {cv_mean} and the R2 score for Training is {training} and R2 score for the Testing is{testing}')
    print('\n')

In [ ]:
# as we see all are giving same result so out model is best model without overfitting 

# Training Score for Linear Regression = 0.9907160861218112
# Testing Score for Linear Regression =  0.9954846871242079

# RandomForestRegressor

In [ ]:
# lets try another model 

# RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor

In [ ]:
# define parameters
parameters={'criterion':['mse','mae','poisson'],
           'max_features':['auto','sqrt','log2'],
           'min_samples_split':[1,11],
           'max_depth':[1,15],
           'min_samples_leaf':[1,7]}

In [ ]:
rf=RandomForestRegressor()
clf=GridSearchCV(rf,parameters)
clf.fit(X_train,y_train)

In [ ]:
#print best parameters
print(clf.best_params_)

In [ ]:
from sklearn.metrics import r2_score

In [ ]:
#reassign best parameters
rf=RandomForestRegressor(criterion= 'mse', max_depth= 15, max_features= 'auto', min_samples_leaf= 1, min_samples_split= 11)
rf.fit(X_train,y_train)
print ('Training R2 Score: ' ,rf.score(X_train,y_train)*100)
pred_decision=rf.predict(X_test)

rfs = r2_score(y_test,pred_decision)
print('Testing R2 Score:' , rfs*100)



In [ ]:
# Training Score for RandomForestRegressor = 95.58203290686926
# Testing Score for RandomForestRegressor =  89.56146461532899

# KNeighborsRegressor

In [ ]:
# lets try another model 

#KNeighborsRegressor

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsRegressor

In [ ]:
# define parameters
parameterss={'algorithm':['kd_tree' , 'brute'],
           'leaf_size':[3,5,10,15,30],
           'n_neighbors':[3,15,7,50,20]}

In [ ]:
knn=KNeighborsRegressor()
clf=clf=GridSearchCV(knn,parameterss)
clf.fit(X_train,y_train)

In [ ]:
#print best parameters
print(clf.best_params_)

In [ ]:
#reassign best parameters
knn=KNeighborsRegressor(algorithm= 'kd_tree', leaf_size= 3, n_neighbors= 7)
knn.fit(X_train,y_train)
print ('Training R2 Score: ' ,knn.score(X_train,y_train)*100)
pred_decision=rf.predict(X_test)

rfs = r2_score(y_test,pred_decision)
print('Testing R2 Score:' , rfs*100)



In [ ]:
# Training Score for KNeighborsRegressor = 91.1742654070661
# Testing Score for KNeighborsRegressor = 89.56146461532899

In [ ]:
# thus we conclude from Top 3 model we select Linear Regreesion for this perticular model beacuse it score is best among 3 models
# Training Score for Linear Regression = 0.9907160861218112
# Testing Score for Linear Regression =  0.9954846871242079